In [1]:
# Cell 1 - imports and the RandomForestModel class
import numpy as np
import pandas as pd
import glob
import os
import joblib
import matplotlib.pyplot as plt

# sklearn imports used later
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, average_precision_score, confusion_matrix, ConfusionMatrixDisplay
)
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Paste your RandomForestModel class here (exact code you provided).
# To avoid duplication I assume you paste the entire class block you shared earlier.
# For example:
class RandomForestModel:
    def __init__(self, n_estimators=100, random_state=42):
        self.model = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state)
        self.scaler = StandardScaler()
        self.is_trained = False
    def preprocess_data(self, X):
        X_scaled = self.scaler.fit_transform(X)
        return X_scaled
    def predict(self, X):
        if not self.is_trained:
            raise Exception("Model is not trained yet.")
        X_scaled = self.scaler.transform(X)
        return self.model.predict(X_scaled)
    def predict_proba(self, X):
        if not self.is_trained:
            raise Exception("Model is not trained yet.")
        X_scaled = self.scaler.transform(X)
        return self.model.predict_proba(X_scaled)[:, 1]
    def evaluate(self, X, y):
        y_pred = self.predict(X)
        y_proba = self.predict_proba(X)
        metrics = {
            "accuracy": accuracy_score(y, y_pred),
            "balanced_accuracy": balanced_accuracy_score(y, y_pred),
            "precision": precision_score(y, y_pred),
            "recall": recall_score(y, y_pred),
            "f1_score": f1_score(y, y_pred),
            "roc_auc": roc_auc_score(y, y_proba),
            "average_precision": average_precision_score(y, y_proba)
        }
        return metrics
    def save_model(self, file_path):
        joblib.dump({
            'model': self.model,
            'scaler': self.scaler
        }, file_path)
    def load_model(self, file_path):
        data = joblib.load(file_path)
        self.model = data['model']
        self.scaler = data['scaler']
        self.is_trained = True
    def hyperparameter_tuning(self, X, y, param_grid, cv=10):
        X_scaled = self.preprocess_data(X)
        grid_search = GridSearchCV(estimator=self.model, param_grid=param_grid, cv=cv, scoring='f1', n_jobs=-1)
        grid_search.fit(X_scaled, y)
        self.model = grid_search.best_estimator_
        self.is_trained = True
        return grid_search.best_params_
    def train(self, X, y):
        X_scaled = self.preprocess_data(X)
        self.model.fit(X_scaled, y)
        self.is_trained = True
    def plots(self, X, y, output_dir):
        if not self.is_trained:
            raise Exception("Model is not trained yet.")
        y_proba = self.predict_proba(X)
        fpr, tpr, _ = roc_curve(y, y_proba)
        plt.figure()
        plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc_score(y, y_proba))
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic')
        plt.legend(loc="lower right")
        plt.savefig(os.path.join(output_dir, 'roc_curve.png'))
        plt.close()
        precision, recall, _ = precision_recall_curve(y, y_proba)
        plt.figure()
        plt.plot(recall, precision, label='Precision-Recall curve (area = %0.2f)' % average_precision_score(y, y_proba))
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curve')
        plt.legend(loc="lower left")
        plt.savefig(os.path.join(output_dir, 'precision_recall_curve.png'))
        plt.close()
    def confusion_matrix_plot(self, X, y, output_dir):
        if not self.is_trained:
            raise Exception("Model is not trained yet.")
        y_pred = self.predict(X)
        cm = confusion_matrix(y, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot(cmap=plt.cm.Blues)
        plt.title('Confusion Matrix')
        plt.savefig(os.path.join(output_dir, 'confusion_matrix.png'))
        plt.close()


In [8]:
# Cell 2 - user settings (edit these paths)
ttest_csv = r"significant_cpgs_ttest.csv"         # CSV listing CpG names (from t-test)
data_folder = r"C:\Users\Stuti\Desktop\Projects\CGP\sample_batches"             # folder containing the 188 csv files
label_col = "parkinsons_disease"               # label column name in the sample CSVs
cpg_colname = "CpG_Site"                       # column name in ttest csv that contains CpG IDs
n_top_features = None                          # if None use all CpGs in ttest file; otherwise use top N
output_dir = r"rf_output"
os.makedirs(output_dir, exist_ok=True)


In [9]:
# Cell 3 - read t-test selected CpGs
tt = pd.read_csv(ttest_csv)
if cpg_colname not in tt.columns:
    print("Available columns in ttest file:", tt.columns.tolist())
    raise SystemExit("Update cpg_colname variable to the actual column name in your t-test CSV.")

cpg_list = tt[cpg_colname].astype(str).tolist()
if n_top_features is not None:
    cpg_list = cpg_list[:n_top_features]

print(f"Total CpGs to use from t-test file: {len(cpg_list)}")


Total CpGs to use from t-test file: 62045


In [ ]:
# Cell 4 - load only needed columns from each data CSV and concatenate
files = sorted(glob.glob(os.path.join(data_folder, "*.csv")))
print("Found files:", len(files))

chunks = []
labels = []
missing_counts = {}

for i, fpath in enumerate(files, 1):
    try:
        # Build column list to read: the CpGs + label (if present)
        usecols = list(cpg_list) + [label_col]
        # Read with pandas using only required columns (this saves memory)
        df_piece = pd.read_csv(fpath, usecols=lambda c: c in usecols)
    except Exception as e:
        # Some files may not contain all CpGs; read full header then subset
        df_full = pd.read_csv(fpath, nrows=0)
        available = [c for c in cpg_list if c in df_full.columns]
        if label_col not in df_full.columns:
            raise ValueError(f"Label column '{label_col}' not found in file {fpath}.")
        missing = len(cpg_list) - len(available)
        missing_counts[fpath] = missing
        print(f"[{i}/{len(files)}] Warning: {missing} CpGs missing in {os.path.basename(fpath)}. Loading available columns.")
        usecols = available + [label_col]
        df_piece = pd.read_csv(fpath, usecols=usecols)
        # Add any missing CpG columns as NaN so shapes align
        for c in cpg_list:
            if c not in df_piece.columns:
                df_piece[c] = np.nan

    # Reorder columns to the same order as cpg_list then label
    df_piece = df_piece[cpg_list + [label_col]]
    chunks.append(df_piece.dropna(subset=[label_col]))  # drop rows missing label if any

    if i % 20 == 0 or i == len(files):
        print(f"Loaded {i}/{len(files)} files")

# Concatenate all pieces
data_all = pd.concat(chunks, ignore_index=True)
print("Concatenated shape:", data_all.shape)
if missing_counts:
    print("Some files had missing CpG columns. Example counts:", dict(list(missing_counts.items())[:3]))


Found files: 189


In [ ]:
# Cell 5 - separate X and y, keep only numeric features and handle NaNs
X = data_all[cpg_list].select_dtypes(include=[np.number])
y = data_all[label_col].astype(int)

print("Before NA handling - X shape:", X.shape, "y shape:", y.shape)

# Handle NaNs: simple strategy = column-wise mean imputation
X = X.fillna(X.mean())

print("After imputation - X shape:", X.shape)


In [ ]:
# Cell 7 - train/test split and model training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

rf = RandomForestModel(n_estimators=200, random_state=42)
rf.train(X_train, y_train)

# Save the trained model
model_path = os.path.join(output_dir, "random_forest_model.joblib")
rf.save_model(model_path)
print("Model saved to:", model_path)


In [ ]:
# Cell 8 - evaluate and save plots
metrics = rf.evaluate(X_test, y_test)
print("Evaluation metrics on test set:")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# create plots (ROC, PR) and confusion matrix
rf.plots(X_test, y_test, output_dir)
rf.confusion_matrix_plot(X_test, y_test, output_dir)
print("Saved plots to", output_dir)
